In [13]:
import wget
import torch

In [6]:
wget.download("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")


'input.txt'

In [8]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

print(f"La taille du texte est de :{len(text)}")
print(text[:1000])


La taille du texte est de :1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for 

In [11]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(f"La taille du vocabe est {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
La taille du vocabe est 65


In [12]:
stoi = {ch:i for (i,ch) in enumerate(chars)}
itos = {i:ch for (i,ch) in enumerate(chars)}

encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("Salut oue"))
print(decode(encode("Salut oue")))

[31, 39, 50, 59, 58, 1, 53, 59, 43]
Salut oue


In [14]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.type)
print(data[:100])

torch.Size([1115394]) <built-in method type of Tensor object at 0x00000227C16AF890>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


In [17]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [20]:
torch.manual_seed(1337)
batchsize = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-block_size,(batchsize,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x,y

x,y = get_batch("train")

print(x.shape)
print(y.shape)
print(x)
print(y)

torch.Size([4, 8])
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [ ]:
import torch.nn as nn

from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets= None):
        logits = self.token_embedding_table(idx)
        #modification de la shape pour la cross entropy
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets  = targets.view(B*T)

            loss = F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self, idx,max_new_tokens):
        #idx is size (B,T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1) #size (B,1)

            idx = torch.cat((idx,idx_next),dim=1)
        return idx
    

m = BigramLanguageModel(vocab_size)
logits,loss = m(x,y)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [28]:
optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)


In [32]:
batchsize = 32

for steps in range(10000):
    xb,yb = get_batch('train')

    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4373364448547363


In [33]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))



Tailapo belll w anive? nanowalarpe JOFiase ig bo, t ced bere
LLe neset aiemy
I wie yat y mexirourous ceckierelsaru t IAREEroutersay.
AD:
Se m ane! :
Hobsad, s IAle h mexK:
We, makisoung, hall-hithin p wate st--
TE:
Ifur met th hire onsteiahe cour, RDilothay Mube t VOLUKERUS:

ABEXExe s s hpr ug y'd it trr,
I lay:
EOMy athaveanghur amex.
Whes:
F: I

Buru.
Serth'Whth:
AUESI isin. thed thmyo as ELI g.
CHUSisthethellend:
GAnd fo,
CKI:
DULird m wotiotoup-PAny, har lre uf,
HEL:
FLod bery s. angupr,


